In [35]:
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, concatenate
from tensorflow.keras.models import Model


In [2]:
file_id ="1_ouiYlKVS23GiRSu-WEBwOv8pXNc2_VM"
file_name = "ceramic-cracks-dataset-master.zip"
target_folder = "unzipedCeramics"

if not os.path.exists(file_name):
    !gdown --id {file_id} -O {file_name}
else:
    print("Zip dosyası zaten mevcut,indirilmedi")

if not os.path.exists(target_folder):
    print(f"'{target_folder}' oluşturuluyor ve zip dosyası içine açılıyor...")
    !mkdir -p "{target_folder}"
    !unzip -q -n "{file_name}" -d "{target_folder}"
    print("İşlem tamamlandı.")
else:
    print(f"'{target_folder}'zaten mevut zip açılmadı.")
##https://drive.google.com/file/d/1_ouiYlKVS23GiRSu-WEBwOv8pXNc2_VM/view?usp=drive_link

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1_ouiYlKVS23GiRSu-WEBwOv8pXNc2_VM
From (redirected): https://drive.google.com/uc?id=1_ouiYlKVS23GiRSu-WEBwOv8pXNc2_VM&confirm=t&uuid=96d41c92-7dc4-4763-978d-dc32cd882740
To: /kaggle/working/ceramic-cracks-dataset-master.zip
100%|██████████████████████████████████████| 37.9M/37.9M [00:03<00:00, 9.57MB/s]
'unzipedCeramics' oluşturuluyor ve zip dosyası içine açılıyor...
İşlem tamamlandı.


In [46]:
BASE_PATH = '/kaggle/working/unzipedCeramics/ceramic-cracks-dataset-master/'
TRAIN_IMAGE_DIR = os.path.join(BASE_PATH, 'train', 'image')
TRAIN_MASK_DIR = os.path.join(BASE_PATH, 'train', 'label')
TEST_IMAGE_DIR = os.path.join(BASE_PATH, 'test', 'image')

IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 8
EPOCHS = 15 

In [30]:
train_image_paths = sorted([os.path.join(TRAIN_IMAGE_DIR, f) for f in os.listdir(TRAIN_IMAGE_DIR) if f.endswith('.png')])
train_mask_paths = sorted([os.path.join(TRAIN_MASK_DIR, f) for f in os.listdir(TRAIN_MASK_DIR) if f.endswith('.png')])

val_image_paths = sorted([os.path.join(VAL_IMAGE_DIR, f) for f in os.listdir(VAL_IMAGE_DIR) if f.endswith('.png')])


print(f"Eğitim için bulunan resim/maske çifti: {len(train_image_paths)}")
print(f"Doğrulama için bulunan resim/maske çifti: {len(val_image_paths)}")

Eğitim için bulunan resim/maske çifti: 100
Doğrulama için bulunan resim/maske çifti: 67


In [47]:
train_image_paths = sorted([os.path.join(TRAIN_IMAGE_DIR, f) for f in os.listdir(TRAIN_IMAGE_DIR) if f.endswith('.png')])
train_mask_paths = sorted([os.path.join(TRAIN_MASK_DIR, f) for f in os.listdir(TRAIN_MASK_DIR) if f.endswith('.png')])

print(f"Eğitim için {len(train_image_paths)} adet resim ve maske bulundu.")

def load_data_into_memory(image_paths, mask_paths):
    images_list = []
    masks_list = []
    
    print(f"{len(image_paths)} adet görüntü ve maske hafızaya yükleniyor...")
    
    for img_path, mask_path in zip(image_paths, mask_paths):
        img = Image.open(img_path).convert('RGB')
        img = img.resize((IMG_WIDTH, IMG_HEIGHT))
        images_list.append(np.array(img) / 255.0)
        
        mask = Image.open(mask_path).convert('L')
        mask = mask.resize((IMG_WIDTH, IMG_HEIGHT), resample=Image.NEAREST)
        mask = np.array(mask)
        mask = np.expand_dims(mask, axis=-1)
        masks_list.append((mask > 0).astype(np.float32))

    return np.array(images_list), np.array(masks_list)

# Bütün train setini yüklüyoruz
X_train, y_train = load_data_into_memory(train_image_paths, train_mask_paths)

print("\nVeri yükleme tamamlandı.")
print(f"X_train (resimler) şekli: {X_train.shape}")
print(f"y_train (maskeler) şekli: {y_train.shape}")

Eğitim için 100 adet resim ve maske bulundu.
100 adet görüntü ve maske hafızaya yükleniyor...

Veri yükleme tamamlandı.
X_train (resimler) şekli: (100, 224, 224, 3)
y_train (maskeler) şekli: (100, 224, 224, 1)


In [48]:
def build_unet_with_mobilenetv2():
    base_model = tf.keras.applications.MobileNetV2(input_shape=[IMG_HEIGHT, IMG_WIDTH, 3], include_top=False, weights='imagenet')
    layer_names = [
        'block_1_expand_relu', 'block_3_expand_relu', 'block_6_expand_relu', 
        'block_13_expand_relu', 'block_16_project',
    ]
    base_model_outputs = [base_model.get_layer(name).output for name in layer_names]
    encoder = Model(inputs=base_model.input, outputs=base_model_outputs)
    encoder.trainable = False
    inputs = Input(shape=[IMG_HEIGHT, IMG_WIDTH, 3])
    skips = encoder(inputs)
    x = skips[-1]
    skips = reversed(skips[:-1])
    for up, skip in zip([512, 256, 128, 64], skips):
        x = Conv2DTranspose(up, 2, strides=2, padding='same')(x)
        x = concatenate([x, skip])
        x = Conv2D(up, 3, padding='same', activation='relu')(x)
        x = Conv2D(up, 3, padding='same', activation='relu')(x)
    x = Conv2DTranspose(32, 2, strides=2, padding='same')(x)
    output = Conv2D(1, 1, activation='sigmoid')(x)
    return Model(inputs=inputs, outputs=output)

model = build_unet_with_mobilenetv2()
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [49]:
print("\nModel eğitimi başlıyor (Doğrulama seti olmadan)...")
history = model.fit(
    X_train, y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS
)


Model eğitimi başlıyor (Doğrulama seti olmadan)...
Epoch 1/15
13/13 ━━━━━━━━━━━━━━━━━━━━ 52s 3s/step - accuracy: 0.8572 - loss: 0.3336
Epoch 2/15
13/13 ━━━━━━━━━━━━━━━━━━━━ 38s 3s/step - accuracy: 0.9631 - loss: 0.1472
Epoch 3/15
13/13 ━━━━━━━━━━━━━━━━━━━━ 38s 3s/step - accuracy: 0.9504 - loss: 0.1474
Epoch 4/15
13/13 ━━━━━━━━━━━━━━━━━━━━ 39s 3s/step - accuracy: 0.9590 - loss: 0.1270
Epoch 5/15
13/13 ━━━━━━━━━━━━━━━━━━━━ 39s 3s/step - accuracy: 0.9524 - loss: 0.1228
Epoch 6/15
13/13 ━━━━━━━━━━━━━━━━━━━━ 39s 3s/step - accuracy: 0.9557 - loss: 0.1173
Epoch 7/15
13/13 ━━━━━━━━━━━━━━━━━━━━ 38s 3s/step - accuracy: 0.9602 - loss: 0.0974
Epoch 8/15
13/13 ━━━━━━━━━━━━━━━━━━━━ 39s 3s/step - accuracy: 0.9549 - loss: 0.1059
Epoch 9/15
13/13 ━━━━━━━━━━━━━━━━━━━━ 38s 3s/step - accuracy: 0.9599 - loss: 0.0976
Epoch 10/15
13/13 ━━━━━━━━━━━━━━━━━━━━ 38s 3s/step - accuracy: 0.9613 - loss: 0.0950
Epoch 11/15
13/13 ━━━━━━━━━━━━━━━━━━━━ 39s 3s/step - accuracy: 0.9598 - loss: 0.0973
Epoch 12/15
13/13 ━━━━